<a href="https://colab.research.google.com/github/ke-wen/ke-wen/blob/main/posters/sample_dataset_Bert_TF_TF-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Reading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/AP_FYP")
!pwd


/content/drive/MyDrive/AP_FYP


In [ ]:
import pandas as pd

In [ ]:
#Load the Dataset
dataset_path = 'data/HateSpeechDatasetBalanced.csv'
data = pd.read_csv(dataset_path)
print('The shape of the dataset is:', data.shape)

The shape of the dataset is: (726119, 2)


There is too much data, so we randomly select 1/10 of it.And apply the best performing model to whole data

In [ ]:
df = data.sample(frac=0.1, random_state=42)
df

,Content,Label
615013,you should be deeply embarrassed... by not ful...,1
75913,do not make me make you fall in love with a bi...,0
523130,trump america is anti immigrant sexual activit...,1
682117,you guys are clearly a pole smoker please get ...,1
395535,oh come along the only reason people like stri...,0
...,...,...
386016,hi just a means cool a k a to say you that the...,0
439019,kiss i am not forgetting to update them its ju...,0
394958,mediation committee oppose per user page attac...,0
308990,pictures or it did not happen got to see so ma...,0


In [ ]:
df.head(10)

,Content,Label
615013,you should be deeply embarrassed... by not ful...,1
75913,do not make me make you fall in love with a bi...,0
523130,trump america is anti immigrant sexual activit...,1
682117,you guys are clearly a pole smoker please get ...,1
395535,oh come along the only reason people like stri...,0
63523,did you just use it means cool a k a batty are...,0
299499,more versions according to there are some more...,0
270906,thank you for reverting my edit without postin...,0
181806,you are going to claim that heating up methane...,0
312861,tweeters have a fabulous weekend paparazzi,0


In [ ]:
df["Label"].value_counts()

,count
Label,
1,36483
0,36129


2. Exploratory data analysis

In [ ]:
print(df.isna().sum())

Content    0
Label      0
dtype: int64


In [ ]:
!pip install transformers torch scikit-learn

4. Train - Validation Split

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(df["Content"], df['Label'], test_size=0.1, random_state=42)

Use BERT for text embeddings:

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BERT-based Classifier
# Load BERT tokenizer and model, move to GPU
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)

# Tokenize and encode the text data, move to GPU
max_length = 128
X_train_tokens = tokenizer(list(train_data), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)
X_test_tokens = tokenizer(list(test_data), truncation=True, padding=True, max_length=max_length, return_tensors="pt", add_special_tokens=True).to(device)

# Calculate BERT embeddings for the text data
def get_bert_embeddings(tokens):
    embeddings = []
    for i in tqdm(range(len(tokens['input_ids']))):
        with torch.no_grad():
            output = bert_model(input_ids=tokens['input_ids'][i].unsqueeze(0), attention_mask=tokens['attention_mask'][i].unsqueeze(0))
        embeddings.append(output[0].squeeze().mean(dim=0).cpu().numpy())
    return embeddings


X_train_bert_embeddings = get_bert_embeddings(X_train_tokens)
X_test_bert_embeddings = get_bert_embeddings(X_test_tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 7262/7262 [01:18<00:00, 92.37it/s] 


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Try logistic regression, naive Bayes and knn models
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_bert_embeddings, train_labels)

NB = GaussianNB()
NB.fit(X_train_bert_embeddings, train_labels)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_bert_embeddings, train_labels)


LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test data
predictions = knn.predict(X_test_bert_embeddings)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

predictions = NB.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Naive Bais Accuracy: {accuracy * 100:.2f}%')


predictions = lr.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Logistic Regression Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 79.19%
Naive Bais Accuracy: 70.99%
Logistic Regression Accuracy: 80.65%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test data
predictions = knn.predict(X_test_bert_embeddings)
# Create a classification report
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

# Create a confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)

# Print the classification report and confusion matrix
print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)

predictions = NB.predict(X_test_bert_embeddings)
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])

conf_matrix = confusion_matrix(test_labels, predictions)

print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)

predictions = lr.predict(X_test_bert_embeddings)
class_report = classification_report(test_labels, predictions, target_names=['negative', 'positive'])
conf_matrix = confusion_matrix(test_labels, predictions)

print("Classification Report:")
print(class_report)

print("\nConfusion Matrix:")
print(conf_matrix)




Classification Report:
              precision    recall  f1-score   support

    negative       0.81      0.76      0.79      3621
    positive       0.78      0.82      0.80      3641

    accuracy                           0.79      7262
   macro avg       0.79      0.79      0.79      7262
weighted avg       0.79      0.79      0.79      7262


Confusion Matrix:
[[2759  862]
 [ 649 2992]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.77      0.60      0.67      3621
    positive       0.67      0.82      0.74      3641

    accuracy                           0.71      7262
   macro avg       0.72      0.71      0.71      7262
weighted avg       0.72      0.71      0.71      7262


Confusion Matrix:
[[2165 1456]
 [ 651 2990]]
Classification Report:
              precision    recall  f1-score   support

    negative       0.82      0.79      0.80      3621
    positive       0.80      0.82      0.81      3641

    accuracy         

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# Support Vector Machine (SVM)
svm = SVC(kernel='linear', max_iter=1000, random_state=42)
svm.fit(X_train_bert_embeddings, train_labels)
svm_predictions = svm.predict(X_test_bert_embeddings)
svm_accuracy = accuracy_score(test_labels, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy * 100:.2f}%')
print("SVM Classification Report:")
print(classification_report(test_labels, svm_predictions))

# Gradient Boosting (GB)
gb = GradientBoostingClassifier(n_estimators=30, learning_rate=0.05, max_depth=2, random_state=42)
gb.fit(X_train_bert_embeddings, train_labels)
gb_predictions = gb.predict(X_test_bert_embeddings)
gb_accuracy = accuracy_score(test_labels, gb_predictions)
print(f'Gradient Boosting Accuracy: {gb_accuracy * 100:.2f}%')
print("Gradient Boosting Classification Report:")
print(classification_report(test_labels, gb_predictions))

# Random Forest (RF)
rf = RandomForestClassifier(n_estimators=30, random_state=42)
rf.fit(X_train_bert_embeddings, train_labels)
rf_predictions = rf.predict(X_test_bert_embeddings)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print(f'Random Forest Accuracy: {rf_accuracy * 100:.2f}%')
print("Random Forest Classification Report:")
print(classification_report(test_labels, rf_predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 56.16%
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.50      0.53      3621
           1       0.56      0.63      0.59      3641

    accuracy                           0.56      7262
   macro avg       0.56      0.56      0.56      7262
weighted avg       0.56      0.56      0.56      7262

Gradient Boosting Accuracy: 73.37%
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.67      0.71      3621
           1       0.71      0.80      0.75      3641

    accuracy                           0.73      7262
   macro avg       0.74      0.73      0.73      7262
weighted avg       0.74      0.73      0.73      7262

Random Forest Accuracy: 76.74%
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.74      0.76      3621
           1       0.76      0.79      0.77  

Tuning:Try More iterations for Logistic Regression

In [ ]:
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_bert_embeddings, train_labels)
predictions = lr.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'Logistic Regression Accuracy: {accuracy * 100:.2f}%')

Logistic Regression Accuracy: 80.65%


No significant improvement

try diferent k

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_bert_embeddings, train_labels)
predictions = knn.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 77.94%


In [ ]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_bert_embeddings, train_labels)
predictions = knn.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 79.34%


In [ ]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train_bert_embeddings, train_labels)
predictions = knn.predict(X_test_bert_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

KNN Accuracy: 79.48%


No significant improvement

More iteration for SVM

In [ ]:
svm = SVC(kernel='linear', max_iter=10000, random_state=42)
svm.fit(X_train_bert_embeddings, train_labels)
svm_predictions = svm.predict(X_test_bert_embeddings)
svm_accuracy = accuracy_score(test_labels, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy * 100:.2f}%')
print("SVM Classification Report:")
print(classification_report(test_labels, svm_predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 61.97%
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.55      0.59      3621
           1       0.61      0.69      0.65      3641

    accuracy                           0.62      7262
   macro avg       0.62      0.62      0.62      7262
weighted avg       0.62      0.62      0.62      7262



The accuracy is slightly improved, but it is far inferior to other models

Change feature extractor to TF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Use Count Vectorizer (TF) to transform the text data
vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tf = vectorizer.fit_transform(train_data)
X_test_tf = vectorizer.transform(test_data)

## Try logistic regression, naive Bayes, KNN, SVM, Gradient Boosting, and Random Forest models

# 1. K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_tf, train_labels)
knn_predictions = knn.predict(X_test_tf)
knn_accuracy = accuracy_score(test_labels, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy * 100:.2f}%')
print("KNN Classification Report:")
print(classification_report(test_labels, knn_predictions))

# 2. Naive Bayes (MultinomialNB)
nb = MultinomialNB()
nb.fit(X_train_tf, train_labels)
nb_predictions = nb.predict(X_test_tf)
nb_accuracy = accuracy_score(test_labels, nb_predictions)
print(f'Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%')
print("Naive Bayes Classification Report:")
print(classification_report(test_labels, nb_predictions))

# 3. Logistic Regression
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_tf, train_labels)
lr_predictions = lr.predict(X_test_tf)
lr_accuracy = accuracy_score(test_labels, lr_predictions)
print(f'Logistic Regression Accuracy: {lr_accuracy * 100:.2f}%')
print("Logistic Regression Classification Report:")
print(classification_report(test_labels, lr_predictions))

# 4. Support Vector Machine (SVM)
svm = SVC(kernel='linear', max_iter=1000)
svm.fit(X_train_tf, train_labels)
svm_predictions = svm.predict(X_test_tf)
svm_accuracy = accuracy_score(test_labels, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy * 100:.2f}%')
print("SVM Classification Report:")
print(classification_report(test_labels, svm_predictions))

# 5. Gradient Boosting (GB)
gb = GradientBoostingClassifier(n_estimators=30, learning_rate=0.05, max_depth=2, random_state=42)
gb.fit(X_train_tf, train_labels)
gb_predictions = gb.predict(X_test_tf)
gb_accuracy = accuracy_score(test_labels, gb_predictions)
print(f'Gradient Boosting Accuracy: {gb_accuracy * 100:.2f}%')
print("Gradient Boosting Classification Report:")
print(classification_report(test_labels, gb_predictions))

# 6. Random Forest (RF)
rf = RandomForestClassifier(n_estimators=30, random_state=42)
rf.fit(X_train_tf, train_labels)
rf_predictions = rf.predict(X_test_tf)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print(f'Random Forest Accuracy: {rf_accuracy * 100:.2f}%')
print("Random Forest Classification Report:")
print(classification_report(test_labels, rf_predictions))


KNN Accuracy: 66.74%
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.72      0.68      3621
           1       0.69      0.61      0.65      3641

    accuracy                           0.67      7262
   macro avg       0.67      0.67      0.67      7262
weighted avg       0.67      0.67      0.67      7262

Naive Bayes Accuracy: 76.38%
Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.62      0.72      3621
           1       0.71      0.90      0.79      3641

    accuracy                           0.76      7262
   macro avg       0.79      0.76      0.76      7262
weighted avg       0.79      0.76      0.76      7262

Logistic Regression Accuracy: 80.07%
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      3621
           1       0.79      0.82      0.80  

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 58.36%
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.23      0.35      3621
           1       0.55      0.94      0.69      3641

    accuracy                           0.58      7262
   macro avg       0.67      0.58      0.52      7262
weighted avg       0.66      0.58      0.52      7262

Gradient Boosting Accuracy: 64.69%
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.47      0.57      3621
           1       0.61      0.82      0.70      3641

    accuracy                           0.65      7262
   macro avg       0.67      0.65      0.64      7262
weighted avg       0.67      0.65      0.64      7262

Random Forest Accuracy: 78.05%
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.77      0.78      3621
           1       0.78      0.79      0.78  

Change feature extractor to TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Use TF-IDF to transform the text data
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(train_data)
X_test_tfidf = vectorizer.transform(test_data)

## Try logistic regression, naive Bayes, knn, SVM, Gradient Boosting, and Random Forest models

# 1. K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_tfidf, train_labels)
knn_predictions = knn.predict(X_test_tfidf)
knn_accuracy = accuracy_score(test_labels, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy * 100:.2f}%')
print("KNN Classification Report:")
print(classification_report(test_labels, knn_predictions))

# 2. Naive Bayes (MultinomialNB)
nb = MultinomialNB()
nb.fit(X_train_tfidf, train_labels)
nb_predictions = nb.predict(X_test_tfidf)
nb_accuracy = accuracy_score(test_labels, nb_predictions)
print(f'Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%')
print("Naive Bayes Classification Report:")
print(classification_report(test_labels, nb_predictions))

# 3. Logistic Regression
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train_tfidf, train_labels)
lr_predictions = lr.predict(X_test_tfidf)
lr_accuracy = accuracy_score(test_labels, lr_predictions)
print(f'Logistic Regression Accuracy: {lr_accuracy * 100:.2f}%')
print("Logistic Regression Classification Report:")
print(classification_report(test_labels, lr_predictions))

# 4. Support Vector Machine (SVM)
svm = SVC(kernel='linear', max_iter=1000, random_state=42)
svm.fit(X_train_tfidf, train_labels)
svm_predictions = svm.predict(X_test_tfidf)
svm_accuracy = accuracy_score(test_labels, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy * 100:.2f}%')
print("SVM Classification Report:")
print(classification_report(test_labels, svm_predictions))

# 5. Gradient Boosting (GB)
gb = GradientBoostingClassifier(n_estimators=30, learning_rate=0.05, max_depth=2, random_state=42)
gb.fit(X_train_tfidf, train_labels)
gb_predictions = gb.predict(X_test_tfidf)
gb_accuracy = accuracy_score(test_labels, gb_predictions)
print(f'Gradient Boosting Accuracy: {gb_accuracy * 100:.2f}%')
print("Gradient Boosting Classification Report:")
print(classification_report(test_labels, gb_predictions))

# 6. Random Forest (RF)
rf = RandomForestClassifier(n_estimators=30, random_state=42)
rf.fit(X_train_tfidf, train_labels)
rf_predictions = rf.predict(X_test_tfidf)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print(f'Random Forest Accuracy: {rf_accuracy * 100:.2f}%')
print("Random Forest Classification Report:")
print(classification_report(test_labels, rf_predictions))


KNN Accuracy: 56.87%
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.55      0.56      3621
           1       0.57      0.59      0.58      3641

    accuracy                           0.57      7262
   macro avg       0.57      0.57      0.57      7262
weighted avg       0.57      0.57      0.57      7262

Naive Bayes Accuracy: 77.28%
Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.69      0.75      3621
           1       0.73      0.86      0.79      3641

    accuracy                           0.77      7262
   macro avg       0.78      0.77      0.77      7262
weighted avg       0.78      0.77      0.77      7262

Logistic Regression Accuracy: 80.18%
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      3621
           1       0.80      0.81      0.80  

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 60.81%
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.47      0.55      3621
           1       0.59      0.74      0.66      3641

    accuracy                           0.61      7262
   macro avg       0.62      0.61      0.60      7262
weighted avg       0.62      0.61      0.60      7262

Gradient Boosting Accuracy: 65.67%
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.47      0.58      3621
           1       0.61      0.84      0.71      3641

    accuracy                           0.66      7262
   macro avg       0.68      0.66      0.64      7262
weighted avg       0.68      0.66      0.64      7262

Random Forest Accuracy: 77.72%
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.77      0.77      3621
           1       0.77      0.79      0.78  

BERT embeddings are the best feature extractor. Next, I will apply it to the entire dataset.